In [1]:
#%pip install "dask[diagnostics]" --upgrade  
#%pip install "dask[distributed]" --upgrade result_HLJ.csv
#%pip install chardet
#%pip install openpyxl

In [2]:
# set dask Client config
import dask.dataframe as dd
from dask.distributed import Client
import dask.config
import pandas as pd

global dtype
global na_value
global province
client = Client(n_workers=14, threads_per_worker=1, memory_limit='32GB')

# set dtype & column need to set N/A value
dtype = {'apntype': str,
         'avg': str,
         'city': str,
         'dropped': float,
         'max': float,
         'min': float,
         'netstandard': str,
         'nettype': float,
         'province': str,
         'rsrp': float,
         'snr': float,
         'stddev': float}

na_value = ['dropped',
            'max',
            'min',
            'nettype',
            'rsrp',
            'snr',
            'stddev']

In [3]:
# df.head(5)
def drop_reason(df, df_drop, rqst: str) -> dask.dataframe.DataFrame:
    # @ ssss
    df_grouped = df.groupby(rqst).size(
    ).compute().reset_index(name='总数')

    dfdrop_grouped = df_drop.groupby(rqst).size(
    ).compute().reset_index(name='拨测失败数')

    # 合并两个分组结果
    merged = df_grouped.merge(dfdrop_grouped, on=rqst, how='left')

    # 计算比例
    merged['失败率'] = merged['拨测失败数'] / merged['总数']
    return merged


# def 5g_reason(df, df_drop, rqst: str) -> dask.dataframe.DataFrame:

def load(province: str):
    src_file_path = 'result_'+province+'.csv'
    df = dd.read_csv(src_file_path, sample_rows=200,
                     dtype=dtype, na_values=na_value,
                     encoding='gb2312', encoding_errors='replace')

    return df


def drop_analyze(df):

    # get drop line
    df_drop = df.loc[df['dropped'] == 1.0]
    df_drop = df_drop.persist()

    # open excel writer
    with pd.ExcelWriter(province+'_drop_reson.xlsx', mode='w') as writer:
        # analyze drop reason with opt_list reasons
        opt_list = ['city', 'apntype', 'host', 'apntype']
        for opt in opt_list:
            reason = drop_reason(df, df_drop, opt).sort_values(by='失败率',
                                                               ascending=False)
            reason.to_excel(writer, sheet_name=opt, index=False)

        # analyze drop reason with cellid top 200
        cell_reason = drop_reason(df, df_drop, 'cellid').sort_values(by='拨测失败数',
                                                                     ascending=False).head(200)
        cell_reason.to_excel(writer, sheet_name='cellid', index=False)

In [4]:

province_list=['LN','SC','NM','JL','HLJ']
# province_list=['XZ']
print("连通率分析：")
for province in province_list:
    df=load(province)
    drop_analyze(df)
    print(province+' done')
print('all done')

连通率分析：
LN done
SC done
NM done
JL done
HLJ done


2024-05-31 11:05:37,549 - distributed.shuffle._scheduler_plugin - WARNING - Shuffle f02036b4075a0abbf180c5cc72f43f06 initialized by task ('shuffle-transfer-f02036b4075a0abbf180c5cc72f43f06', 4) executed on worker tcp://127.0.0.1:42327
2024-05-31 11:05:40,065 - distributed.shuffle._scheduler_plugin - WARNING - Shuffle f02036b4075a0abbf180c5cc72f43f06 deactivated due to stimulus 'task-finished-1717124740.063495'
2024-05-31 11:05:40,160 - distributed.shuffle._scheduler_plugin - WARNING - Shuffle 19e45065bae725aec713c6f5ec2fe2a2 initialized by task ('shuffle-transfer-19e45065bae725aec713c6f5ec2fe2a2', 5) executed on worker tcp://127.0.0.1:41973
2024-05-31 11:05:40,789 - distributed.shuffle._scheduler_plugin - WARNING - Shuffle 19e45065bae725aec713c6f5ec2fe2a2 deactivated due to stimulus 'task-finished-1717124740.7888663'
2024-05-31 11:05:43,085 - distributed.shuffle._scheduler_plugin - WARNING - Shuffle 3e27bb4fdd0c03c9882ec5dbaccbd4fc initialized by task ('shuffle-transfer-3e27bb4fdd0c03c